# 🚀 Real-Time Grid Intelligence Installer
This notebook orchestrates the end-to-end deployment of **Real-Time Grid Intelligence** solution assets into the current Microsoft Fabric workspace using the `fabric-launcher` library.

## This notebook performs the following tasks:
1. **📦 Package Installation**: Install required libraries and dependencies
1. **🚀 Deployment**: download source code, deploy Fabric items, load reference Data
1. **✅ Post-Deployment Tasks**: Complete post-deployment configuration tasks.


## 📦 Package Installation

In [ ]:
%pip install "builtin/fabric_launcher-0.3.0-py3-none-any.whl" --quiet
notebookutils.session.restartPython()

## 🚀 Deployment
Download source code, deploy Fabric items, load reference Data

In [ ]:
# Define user-configurable parameters
DEFAULT_API_ROOT_URL = "https://api.fabric.microsoft.com" #Default is https://api.fabric.microsoft.com, but may vary depending on your environment
DEBUG = False

In [ ]:
import notebookutils
from fabric_launcher import FabricLauncher

launcher = FabricLauncher(notebookutils)

# Deploy solution with data folders
launcher.download_and_deploy(
    repo_owner="slavatrofimov",
    repo_name="Real-Time-Grid-Intelligence-with-Microsoft-Fabric",
    workspace_folder="workspace",
    allow_non_empty_workspace=False,
    item_type_stages = [["KQLDatabase", "Lakehouse", "Eventhouse"],["Notebook", "KQLDashboard", "KQLQueryset", "DataPipeline", "Eventstream", "SemanticModel", "Report", "Reflex", "DataAgent"]],
    data_folders={"data": "data"},
    lakehouse_name="ReferenceDataLH",
    validate_after_deployment=True,
    generate_report=True
)

## ✅ Post-Deployment Tasks

In [ ]:
## Deploy the Service Area Map item
import sempy.fabric as fabric
from fabric_launcher import create_or_update_fabric_item, get_folder_id_by_name, move_item_to_folder, scan_logical_ids

def deploy_map(item_name="Service Area Map", item_type="Map", item_relative_path="Visualize and Chat/Service Area Map.Map", 
               folder_name="Visualize and Chat", endpoint="maps", repository_directory = launcher.repository_path,
               description="Service Area Map for Real-Time Grid Intelligence solution"):
    """Use fabric_launcher utilities to deploy an additional item: Map"""

    # Initialize Fabric client and workspace
    client = fabric.FabricRestClient()
    workspace_id = fabric.get_workspace_id()

    # Step 1: Scan logical IDs
    print("\n1. Scanning logical IDs in repository...")
    logical_id_map = scan_logical_ids(
        repository_directory=repository_directory, workspace_id=workspace_id, client=client
    )

    # Step 2: Create/update a custom item with logical ID replacement
    print("\n2. Creating/updating custom Fabric item...")
    item_id = create_or_update_fabric_item(
        item_name=item_name,
        item_type=item_type,
        item_relative_path=item_relative_path,
        repository_directory=repository_directory,
        workspace_id=workspace_id,
        client=client,
        endpoint=endpoint,
        logical_id_map=logical_id_map,
        description=description,
    )
    print(f"   Item ID: {item_id}")

    # Step 3: Move item to appropriate folder
    print("\n3. Moving item to target folder...")
    success = move_item_to_folder(
        item_name=item_name,
        item_type=item_type,
        folder_name=folder_name,
        workspace_id=workspace_id,
        client=client
    )

    if success:
        print("✅ Map created successfully")

# Perform the deployment of the Service Area Map
deploy_map()

In [ ]:
# Move the KQL Database to the same subfolder as the Eventhouse to compensate for the unexpected placement of the item in the initial deployment
client = fabric.FabricRestClient()
workspace_id = fabric.get_workspace_id()

move_item_to_folder(
        item_name="PowerUtilitiesEH",
        item_type="Eventhouse",
        folder_name="Store and Query",
        workspace_id=workspace_id,
        client=client
    )
print("✅ Eventhouse moved successfully!")

In [ ]:
# Run Post-Deployment Config notebook to finalize setup
result = launcher.run_notebook_synchronous(
    notebook_name="Post-Deployment Config",
    parameters={},
    timeout_seconds=3600
)

In [ ]:
# Load AMI Reference Data Simulation notebook to populate reference data
result = launcher.run_notebook_synchronous(
    notebook_name="AMI Reference Data Simulation",
    parameters={},
    timeout_seconds=3600
)

In [ ]:
from fabric_launcher.post_deployment_utils import (
    get_kusto_query_uri,
    create_accelerated_shortcut_in_kql_db
)

# ============================================================================
# Configuration Parameters
# ============================================================================
target_workspace_id = fabric.resolve_workspace_id()
target_eventhouse_name = 'PowerUtilitiesEH'
target_kql_db_name = 'PowerUtilitiesEH'
source_workspace_id = target_workspace_id 
source_item_id = fabric.resolve_item_id('ReferenceDataLH', 'Lakehouse')

# ============================================================================
# Create shortcuts for required tables
# ============================================================================
tables = ['feeders', 'meters', 'substations', 'transformers']

for table in tables:
    source_path = f"Tables/{table}" 
    target_shortcut_name = table.capitalize()
    
    print(f"\n{'='*70}")
    print(f"Creating accelerated shortcut for table: {table}")
    print(f"{'='*70}")
    
    try:
        create_accelerated_shortcut_in_kql_db(
            notebookutils=notebookutils,
            target_workspace_id=target_workspace_id,
            target_eventhouse_name=target_eventhouse_name,
            target_kql_db_name=target_kql_db_name,
            target_shortcut_name=target_shortcut_name,
            source_workspace_id=source_workspace_id,
            source_item_id=source_item_id,
            source_path=source_path
        )
        print(f"✅ Successfully created accelerated shortcut for '{table}'")
        
    except Exception as e:
        print(f"❌ Failed to create shortcut for '{table}': {str(e)}")
        # Continue with next table instead of stopping
        continue

print(f"\n{'='*70}")
print("✅ Shortcut creation process completed")
print(f"{'='*70}")